In [2]:
import boto3
import yaml
import sqlite3

with open("config.yml", "r") as infile:
    secret = yaml.safe_load(infile)
    AWS_ACCESS_KEY = secret["AWS_ACCESS_KEY"]
    AWS_SECRET_KEY = secret["AWS_SECRET_KEY"]
bucket = "noaa-nexrad-level2"

In [18]:
year_list = []
year_day_list = []
year_day_hour = []
year_day_hour_station_code = set()

client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

prefix = ""
result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
for o in result.get("CommonPrefixes"):
    year_list.append(o.get("Prefix"))

for year in year_list:
    prefix = year
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
    for o in result.get("CommonPrefixes"):
        year_day_list.append(o.get("Prefix"))

for year_day in year_day_list:
    prefix = year_day
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
    for o in result.get("CommonPrefixes"):
        year_day_hour.append(o.get("Prefix"))

for year_day_hour in year_day_hour:
    prefix = year_day_hour
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter="/")
    for o in result.get("CommonPrefixes"):
        meta_data = o.get("Prefix").split("/")
        year_day_hour_station_code.add(
            (meta_data[0], meta_data[1], meta_data[2], meta_data[3])
        )

conn = sqlite3.connect("noaa_goes_date.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS noaa_nexrad_level2_date")
# Create a new table
cursor.execute(
    "CREATE TABLE IF NOT EXISTS noaa_nexrad_level2_date (year text, month text, day text, station text)"
)
for data in year_day_hour_station_code:
    year, month, day, station = data
    cursor.execute(
        "INSERT INTO noaa_nexrad_level2_date VALUES (?, ?, ?, ?)",
        (year, month, day, station),
    )
conn.commit()
conn.close()